In [ ]:
import modulos as md

#lista=md.filtradodemonedas()

lista = ['BCHUSDT', 'XRPUSDT', 'EOSUSDT', 'LTCUSDT', 'TRXUSDT', 'ETCUSDT', 'LINKUSDT', 'XLMUSDT', 'ADAUSDT', 'BNBUSDT', 'ATOMUSDT', 'ALGOUSDT', 
         'KNCUSDT', 'COMPUSDT', 'DOGEUSDT', 'KAVAUSDT', 'WAVESUSDT', 'MKRUSDT', 'SNXUSDT', 'DOTUSDT', 'CRVUSDT', 'SOLUSDT', 'STORJUSDT', 'UNIUSDT', 
         'AVAXUSDT', 'FTMUSDT', 'TOMOUSDT', 'NEARUSDT', 'AAVEUSDT', 'FILUSDT', 'MATICUSDT', 'AXSUSDT', 'SANDUSDT', 'XEMUSDT', 'MANAUSDT', 'LINAUSDT', 
         'MTLUSDT', '1000SHIBUSDT', 'MASKUSDT', 'DYDXUSDT', '1000XECUSDT', 'GALAUSDT', 'FLOWUSDT', 'GMTUSDT', 'APEUSDT', 'OPUSDT', 'INJUSDT', 'LDOUSDT', 
         'APTUSDT', 'FETUSDT', 'RNDRUSDT', 'AGIXUSDT', 'PHBUSDT', 'CFXUSDT', 'STXUSDT', 'LQTYUSDT', 'ARBUSDT', 'SUIUSDT', '1000PEPEUSDT', 'XVGUSDT']


In [2]:
###################################### ESTRATEGIA BB
###################################################################################################################
import modulos as md
lista=['XRPUSDT',	
'EOSUSDT',	
'XTZUSDT',	
'OMGUSDT',	
'CRVUSDT',	
'SUSHIUSDT',
'ENJUSDT',	
'CTKUSDT',	
'1INCHUSDT',
'MANAUSDT',	
'IMXUSDT',	
'MINAUSDT',	
'PERPUSDT',	
'IDUSDT',	
'HFTUSDT',	
'COMBOUSDT',
'MAVUSDT']

#lista=md.lista_de_monedas ()

for symbol in lista:    
    try:
        data = md.sigo_variacion_bitcoin(symbol,timeframe='15m',porc=0.8,ventana=2,tp_flag = True)
        #data = md.estrategia_bb(symbol)
        resultado = md.backtesting(data, plot_flag = False)
        #if (resultado['Profit Factor'] > 1 
        #    and resultado['Return [%]']/resultado['# Trades'] >=0.3 
        #    and resultado['Win Rate [%]']>50):
        print(f"{symbol} - Return [%]: {md.truncate(resultado['Return [%]'],2)}% - # Trades: {resultado['# Trades']} - Profit Factor: {md.truncate(resultado['Profit Factor'],2)}")
    except Exception as falla:
        pass

XRPUSDT - Return [%]: 16.59% - # Trades: 7 - Profit Factor: 2.45
EOSUSDT - Return [%]: 5.89% - # Trades: 7 - Profit Factor: 2.01
XTZUSDT - Return [%]: 5.08% - # Trades: 7 - Profit Factor: 2.42
OMGUSDT - Return [%]: 8.07% - # Trades: 6 - Profit Factor: 6.15
CRVUSDT - Return [%]: 6.31% - # Trades: 7 - Profit Factor: 2.89
SUSHIUSDT - Return [%]: 5.61% - # Trades: 7 - Profit Factor: 2.54
ENJUSDT - Return [%]: 2.28% - # Trades: 7 - Profit Factor: 2.38
CTKUSDT - Return [%]: 2.96% - # Trades: 7 - Profit Factor: 2.04
1INCHUSDT - Return [%]: 4.49% - # Trades: 8 - Profit Factor: 3.72
MANAUSDT - Return [%]: 3.85% - # Trades: 7 - Profit Factor: 2.73
IMXUSDT - Return [%]: 4.64% - # Trades: 7 - Profit Factor: 2.15
MINAUSDT - Return [%]: 6.32% - # Trades: 8 - Profit Factor: 3.33
PERPUSDT - Return [%]: 5.82% - # Trades: 7 - Profit Factor: 4.39
IDUSDT - Return [%]: 5.06% - # Trades: 6 - Profit Factor: 19.22
HFTUSDT - Return [%]: 4.11% - # Trades: 7 - Profit Factor: 3.57
COMBOUSDT - Return [%]: 8.01% - 